In [20]:
import pandas as pd
import numpy as np
import re
import nltk
from nltk.corpus import stopwords
import os
from sklearn import linear_model


In [3]:
df_feat = pd.read_csv('features_validation_complete.csv')
df_feat = df_feat.drop(columns = df_feat.columns[0])
df_emo = pd.read_csv(os.path.join('deezer_mood_detection_dataset','validation.csv'))

In [4]:
df_feat.shape, df_emo.shape

((2712, 16), (3863, 7))

In [5]:
df_feat.head()

,artist,trackname,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,id,time_signature,lyrics
0,CALLA,Strangler,0.527,0.723,11,-6.695,0,0.0260,0.000066,0.6450,0.2480,0.540,139.967,5s0tibKDCoGTyncfK7VKzF,4,Something's gotten into your head\mSay what Yo...
1,The Box Tops,The Letter,0.638,0.428,9,-12.156,0,0.0687,0.252000,0.0000,0.1320,0.901,139.434,6RJK553YhstRzyKA4mug09,4,Gimme a ticket for an \maeroplane\mAin't got t...
2,Chris Whitley,Big Sky Country,0.663,0.437,10,-11.137,1,0.0330,0.503000,0.0000,0.0629,0.598,90.169,3MOHZpSZQmP2f1f30aqpDH,4,Now when this is over\mOver and through\mAnd a...
3,Chris Garneau,Castle-Time,0.695,0.155,7,-13.765,1,0.0371,0.939000,0.5080,0.1030,0.118,112.926,0uGsZC2Yc9OmjGIjSBA4Yp,3,NaN
4,Chris Garneau,Baby's Romance,0.591,0.326,5,-10.393,1,0.0286,0.844000,0.0221,0.1230,0.176,110.075,0fVJW7ozV3cnaBdznrmerO,4,[Verse 1]\mThe baby's sleeping in the crib on ...


In [6]:
# df_feat.iloc[1045]

In [7]:
# df_feat.iloc[2710]


In [8]:
# drop nans
df_feat = df_feat.dropna()

remove everything between brackets (like [chorus])

In [9]:
def preprocess_text(text: str) -> str:
    '''removes everything between square brackets
    deletes \\m lineseperator'''
    text = re.sub("[\[].*?[\]]", " ", text)
    text = re.sub('\\\\m', ' ',text)
    return text

apply to whole df

In [10]:
df_feat['lyrics_cleaned'] = df_feat['lyrics'].apply(lambda x: preprocess_text(x))

In [11]:
df_feat.head()

,artist,trackname,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,id,time_signature,lyrics,lyrics_cleaned
0,CALLA,Strangler,0.527,0.723,11,-6.695,0,0.0260,0.000066,0.64500,0.2480,0.540,139.967,5s0tibKDCoGTyncfK7VKzF,4,Something's gotten into your head\mSay what Yo...,Something's gotten into your head Say what You...
1,The Box Tops,The Letter,0.638,0.428,9,-12.156,0,0.0687,0.252000,0.00000,0.1320,0.901,139.434,6RJK553YhstRzyKA4mug09,4,Gimme a ticket for an \maeroplane\mAin't got t...,Gimme a ticket for an aeroplane Ain't got tim...
2,Chris Whitley,Big Sky Country,0.663,0.437,10,-11.137,1,0.0330,0.503000,0.00000,0.0629,0.598,90.169,3MOHZpSZQmP2f1f30aqpDH,4,Now when this is over\mOver and through\mAnd a...,Now when this is over Over and through And all...
4,Chris Garneau,Baby's Romance,0.591,0.326,5,-10.393,1,0.0286,0.844000,0.02210,0.1230,0.176,110.075,0fVJW7ozV3cnaBdznrmerO,4,[Verse 1]\mThe baby's sleeping in the crib on ...,The baby's sleeping in the crib on top The b...
5,Judy Garland,Over The Rainbow,0.204,0.161,8,-15.191,1,0.0336,0.972000,0.00009,0.1210,0.316,82.009,3wAIcORchxdSkWv6v5AkaU,4,[Verse 1]\mSomewhere over the rainbow\mWay up ...,Somewhere over the rainbow Way up high There...


In [12]:
df_feat.shape

(2558, 17)

In [13]:
df_emo.rename(columns = {'artist_name':'artist','track_name':'trackname','valence':'y_valence', 'arousal': 'y_arousal'},inplace = True)

In [14]:
df_emo.drop(columns = ['dzr_sng_id','MSD_sng_id','MSD_track_id'], inplace = True)

In [15]:
df_emo.head()

,y_valence,y_arousal,artist,trackname
0,-1.176640,-0.314720,CALLA,Strangler
1,0.482478,-0.471437,Gun Club,Sex beat
2,-0.780962,-0.789480,The Box Tops,The Letter
3,-0.767318,0.911361,Chris Whitley,Big Sky Country
4,-0.867375,-0.548259,Chris Garneau,Castle-Time


In [16]:
# df_join = 

In [17]:
# df_emo['trackname'] = df_emo['trac']

In [18]:
merged_data= df_feat.merge(df_emo,how = 'inner',on=["artist","trackname"])


In [19]:
merged_data.head()

,artist,trackname,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,id,time_signature,lyrics,lyrics_cleaned,y_valence,y_arousal
0,CALLA,Strangler,0.527,0.723,11,-6.695,0,0.0260,0.000066,0.64500,0.2480,0.540,139.967,5s0tibKDCoGTyncfK7VKzF,4,Something's gotten into your head\mSay what Yo...,Something's gotten into your head Say what You...,-1.176640,-0.314720
1,The Box Tops,The Letter,0.638,0.428,9,-12.156,0,0.0687,0.252000,0.00000,0.1320,0.901,139.434,6RJK553YhstRzyKA4mug09,4,Gimme a ticket for an \maeroplane\mAin't got t...,Gimme a ticket for an aeroplane Ain't got tim...,-0.780962,-0.789480
2,Chris Whitley,Big Sky Country,0.663,0.437,10,-11.137,1,0.0330,0.503000,0.00000,0.0629,0.598,90.169,3MOHZpSZQmP2f1f30aqpDH,4,Now when this is over\mOver and through\mAnd a...,Now when this is over Over and through And all...,-0.767318,0.911361
3,Chris Garneau,Baby's Romance,0.591,0.326,5,-10.393,1,0.0286,0.844000,0.02210,0.1230,0.176,110.075,0fVJW7ozV3cnaBdznrmerO,4,[Verse 1]\mThe baby's sleeping in the crib on ...,The baby's sleeping in the crib on top The b...,-1.487725,-0.360813
4,Judy Garland,Over The Rainbow,0.204,0.161,8,-15.191,1,0.0336,0.972000,0.00009,0.1210,0.316,82.009,3wAIcORchxdSkWv6v5AkaU,4,[Verse 1]\mSomewhere over the rainbow\mWay up ...,Somewhere over the rainbow Way up high There...,1.071901,0.846830


In [21]:
merged_data.columns


Index(['artist', 'trackname', 'danceability', 'energy', 'key', 'loudness',
       'mode', 'speechiness', 'acousticness', 'instrumentalness', 'liveness',
       'valence', 'tempo', 'id', 'time_signature', 'lyrics', 'lyrics_cleaned',
       'y_valence', 'y_arousal'],
      dtype='object')

In [30]:
X = merged_data[['danceability', 'energy', 'key', 'loudness',
       'mode', 'speechiness', 'acousticness', 'instrumentalness', 'liveness',
       'valence', 'tempo', 'time_signature']]
y = merged_data[['y_valence','y_arousal']]
y_val = merged_data[['y_valence']]
y_aro = merged_data[['y_arousal']]

In [41]:
linear = linear_model.LinearRegression()
linear.fit(X, y)
print(f'PREDICTION TWO DIMENSION Y: {linear.predict(X)}')
print(f'SCORE TWO DIMENSION Y: {linear.score(X, y)}')
print(f'\n{linear.coef_ = }, {linear.intercept_= }')

PREDICTION TWO DIMENSION Y: [[-1.46059258e-01  7.68836451e-02]
 [ 4.18958515e-01  1.66867258e-01]
 [ 1.44341796e-01  6.33487999e-04]
 ...
 [-6.30584817e-01  6.26504839e-01]
 [-7.02096483e-01 -4.92408703e-01]
 [ 4.93204298e-02  1.75404454e-01]]
SCORE TWO DIMENSION Y: 0.17332615995691225

linear.coef_ = array([[ 5.22658613e-01, -4.38802239e-02, -5.95598094e-03,
         1.21920309e-02,  1.46720346e-01, -1.53195071e-01,
         4.80557609e-02, -1.56643733e-01,  2.29792343e-01,
         1.30001026e+00,  9.78902007e-04,  4.34174811e-02],
       [-8.81239376e-01,  1.09047405e+00, -3.78802455e-03,
         8.28265323e-03,  1.06235574e-01,  7.59243786e-01,
        -4.11265518e-03, -3.89448723e-01,  1.52230928e-01,
         7.81477270e-01, -2.13933312e-04,  8.53959978e-02]]), linear.intercept_= array([-1.20729632, -0.68993234])


In [32]:
linear_val = linear_model.LinearRegression()
linear_val.fit(X, y_val)
print(f'PREDICTION VALENCE: {linear_val.predict(X)}')
print(f'SCORE VALENCE: {linear_val.score(X, y_val)}')


PREDICTION VALENCE: [[-0.14605926]
 [ 0.41895852]
 [ 0.1443418 ]
 ...
 [-0.63058482]
 [-0.70209648]
 [ 0.04932043]]
SCORE VALENCE: 0.13676386376204652


In [34]:
linear_aro = linear_model.LinearRegression()
linear_aro.fit(X, y_aro)
print(f'PREDICTION VALENCE: {linear_aro.predict(X)}')
print(f'SCORE VALENCE: {linear_aro.score(X, y_aro)}')


PREDICTION VALENCE: [[ 0.07688365]
 [ 0.16686726]
 [ 0.00063349]
 ...
 [ 0.62650484]
 [-0.4924087 ]
 [ 0.17540445]]
SCORE VALENCE: 0.20988845615177798


In [ ]:
linear = linear_model.LinearRegression()
linear.fit(X, y)
linear.predict(X)
# linear.score(X, y)